# **IMPORTING LIBRARIES**

In [28]:
import numpy as np
import pandas as pd 
import string
import matplotlib.pyplot as plt
import csv
import joblib
import pandas as pd
import tensorflow as tf
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize
import nltk
stop_words = stopwords.words('english')

# **DATA SANITY**

In [2]:
#Loading Data
dataset = pd.read_csv(r'C:\Users\ayush.bajpayee\Desktop\Amazon Hackathon\dataset\train.csv', escapechar='\\',
                      quoting = csv.QUOTE_NONE)

In [4]:
#Do not Tamper this df_original
df_original = dataset.copy()

In [32]:
# #DO NOT RUN
# dataset_to_be_saved = dataset.copy()
# dataset_to_be_saved.to_csv('ModifiedTRAIN.csv', index = False)

In [35]:
dataset.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [38]:
dataset.shape

(2903024, 5)

In [12]:
dataset.columns

Index(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND', 'BROWSE_NODE_ID'], dtype='object')

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2903024 entries, 0 to 2903023
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   TITLE           object
 1   DESCRIPTION     object
 2   BULLET_POINTS   object
 3   BRAND           object
 4   BROWSE_NODE_ID  int64 
dtypes: int64(1), object(4)
memory usage: 110.7+ MB


In [27]:
dataset.isna().mean()

TITLE             0.000024
DESCRIPTION       0.249279
BULLET_POINTS     0.057272
BRAND             0.019544
BROWSE_NODE_ID    0.000000
dtype: float64

In [25]:
dataset.isna().sum()

TITLE                 71
DESCRIPTION       723664
BULLET_POINTS     166263
BRAND              56737
BROWSE_NODE_ID         0
dtype: int64

In [12]:
print(dataset['TITLE'][0])
print('\n')
print(dataset['TITLE'][1])
print('\n')
print(dataset['TITLE'][2])
print('\n')
print(dataset['TITLE'][3])
print('\n')
print(dataset['TITLE'][4])

Pete The Cat Bedtime Blues Doll, 14.5 Inch


The New Yorker NYHM014 Refrigerator Magnet, 2 x 3.5


The Ultimate Self-Sufficiency Handbook: A Complete Guide to Baking, Crafts, Gardening, Preserving Your Harvest, Raising Animals and More


Amway Nutrilite Kids Chewable Iron Tablets (100)


Teacher Planner Company A4 6 Lesson Academic Teacher Planner - Leatherette Pink


In [11]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

[Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches]


[Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height]


Skyhorse Publishing


[Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets]


nan


[Color: Blue,Sleeve: Full Sleeve,Material: Cotton,Neck: Round]


In [22]:
dataset['BULLET_POINTS'].apply(type).value_counts()

<class 'str'>      2736761
<class 'float'>     166263
Name: BULLET_POINTS, dtype: int64

In [17]:
dataset_onlyfloat = dataset.copy()

In [31]:
dataset_onlyfloat['BULLET_POINTS'].fillna(value = 'OTHERS', inplace=True)

In [32]:
dataset_onlyfloat.isna().sum()

TITLE                 71
DESCRIPTION       723664
BULLET_POINTS          0
BRAND              56737
BROWSE_NODE_ID         0
dtype: int64

In [29]:
dataset_onlyfloat[dataset_onlyfloat['BULLET_POINTS'].notna()]

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
...,...,...,...,...,...
2903018,The NXT LVL Liquid Silicone Soft Back Cover Ca...,Product color may slightly vary due to photogr...,[Liquid silicone case compatible with apple pr...,Generic,794481
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933


In [23]:
dataset_onlyfloat[dataset_onlyfloat['BULLET_POINTS'].apply(type) == type('csdv')]

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
...,...,...,...,...,...
2903018,The NXT LVL Liquid Silicone Soft Back Cover Ca...,Product color may slightly vary due to photogr...,[Liquid silicone case compatible with apple pr...,Generic,794481
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933


In [34]:
dataset['BRAND'].apply(type).value_counts()

<class 'str'>      2846287
<class 'float'>      56737
Name: BRAND, dtype: int64

# **DATA PREPROCESSING**

## Dropping Description

In [20]:
dataset.drop(columns='DESCRIPTION', inplace = True)

## Handling Duplicates

In [5]:
#Handling Duplicates
dataset.drop_duplicates(inplace=True)

## Handling **Nulls**

All nulls in all columns, will be replaced by **'OTHERS'**

In [7]:
#Filling Nulls with OTHERS
dataset.fillna('OTHERS', inplace=True)

In [8]:
dataset.isna().sum()

TITLE             0
DESCRIPTION       0
BULLET_POINTS     0
BRAND             0
BROWSE_NODE_ID    0
dtype: int64

## Handling **BULLET_POINTS**

BULLET_POINTS contains a string either **containing** '[]' denoting it is a list of strings or **not containing** '[]' denoting there is only one Bullet Point. Will convert Bullet Points into That specific format

In [9]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

[Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches]


[Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height]


Skyhorse Publishing


[Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets]


OTHERS


[Color: Blue,Sleeve: Full Sleeve,Material: Cotton,Neck: Round]


In [10]:
#Replacing ] and [ with ''
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace('[', '')
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace(']', '')

<ipython-input-10-88901d48649c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace('[', '')
<ipython-input-10-88901d48649c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace(']', '')


In [11]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches


Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height


Skyhorse Publishing


Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets


OTHERS


Color: Blue,Sleeve: Full Sleeve,Material: Cotton,Neck: Round


In [12]:
#Converting BULLET_POINTS from comma seperated values to list containing values
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].map(lambda x: x.split(','))

In [13]:
dataset['BULLET_POINTS'].apply(type).value_counts()

<class 'list'>    2801530
Name: BULLET_POINTS, dtype: int64

## Dumping/Loading

In [14]:
#Dumping
joblib.dump('V1', compress='zlib')

In [2]:
#Loading
dataset = joblib.load('V1')

In [4]:
dataset_original = dataset.copy()
dataset_temp = dataset.iloc[:10000,:].copy()

# **NLP**

## Preprocessing

In [35]:
# Tokenize
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(word_tokenize)

In [29]:
def remove_punc(x):
    x = [''.join(c for c in s if c not in string.punctuation) for s in x]
    x = [s for s in x if s]
    return x

dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(remove_punc)

C:\Users\AYUSH~1.BAJ\AppData\Local\Temp/ipykernel_4940/4247752062.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(remove_punc)


In [38]:
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
9998,Brass Tabla Set Brass Material … (Hammered Bra...,[Brass Bayan (Bass - Left Metal Drum) Head 9 t...,Fasherati,2603
9999,Max Men's Slim Fit Jeans (WI18CDGV08_Mid Blue_28),"[Color Name: Mid Blue, Material: Cotton, Zip f...",Max,40
10000,Gems Gallery Natural & Pure Stone Crystal Bead...,[Rare and Precious gemstone like rose quartz a...,Gems Gallery,1010
10001,Max Men's Checkered Slim Fit Formal Shirt (FBW...,"[Color Name: White, Material: Cotton, Long sle...",Max,1254


In [39]:
X = dataset_temp['TITLE']
y = dataset_temp['BROWSE_NODE_ID']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [43]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [47]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))

accuracy 0.138
Wall time: 548 ms


In [50]:
dataset_temp = dataset.iloc[:100000,:].copy()
dataset_temp.shape

(100000, 4)

In [51]:
X = dataset_temp['TITLE']
y = dataset_temp['BROWSE_NODE_ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))

MemoryError: Unable to allocate 2.63 GiB for an array with shape (353640000,) and data type float64

## Basic Approach

In [98]:
dataset_temp = dataset.iloc[:1000000,:].copy()
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
1019467,Macmerise Metallic Captain America Printed Bac...,"[Ultra vivid, razor sharp colour print techno...",Macmerise,1045
1019468,KD Willmax Stocking Over Knee Soccer Basket Ba...,"[FABRIC, WELL STITCHED and FINISHED: The spor...",KD,27627
1019469,COVERNEW Flip cover for Microsoft Nokia Lumia ...,[Durable Colour],COVERNEW,1295
1019470,DDR3 4 Gb Ram For Gigabyte G1.Sniper 3 (rev. 1...,"[4 Gb DDR 3 Desktop Ram, Un-buffered, Non-ECC...",Bright Solutions,165


In [99]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(dataset_temp['TITLE'], dataset_temp['BROWSE_NODE_ID'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

MemoryError: Unable to allocate 50.0 GiB for an array with shape (750000, 8945) and data type int64

In [21]:
#Transforming Test
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [12]:
%%time

#Train Accuracy
y_pred = clf.predict(X_train_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_train))

accuracy 0.25093333333333334
Wall time: 53.3 s


In [23]:
%%time

#Test Accuracy
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.2306
Wall time: 15.9 s


## Further with Stemming

In [81]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
#Porter Stemmer
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
sno = nltk.stem.SnowballStemmer('english')
# stemmed = [porter.stem(word) for word in tokens]

In [96]:
dataset_temp = dataset.iloc[:100000,:].copy()

In [88]:
#PORTER STEMMING
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(porter.stem)

In [83]:
#Snowball STEMMING
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(sno.stem)

In [89]:
# dataset_temp['TITLE'].apply(tokenizer.tokenize)
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(tokenizer.tokenize)

In [90]:
# dataset_temp['TITLE'].apply(lambda x: [item for item in x if item not in stop_words])
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(lambda x: [item for item in x if item not in stop_words])

In [101]:
%%time
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(dataset_temp['TITLE'], dataset_temp['BROWSE_NODE_ID'], random_state = 0)
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}', stop_words = 'english')
# count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(X_train)
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_counts, y_train)

MemoryError: Unable to allocate 50.0 GiB for an array with shape (750000, 8945) and data type int64

In [ ]:
%%time

#Transforming Test
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [ ]:
%%time

#Train Accuracy
y_pred = clf.predict(X_train_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_train))

In [ ]:
%%time

#Test Accuracy
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))

# DEEP LEARNING

In [7]:
embeddings_index = {}
try:
    f = open('glove.840B.300d.txt', encoding='utf8')
    for line in tqdm(f):
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))
    
except:
    pass

52343it [00:05, 10001.73it/s]


In [8]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [81]:
dataset_temp = dataset.iloc[:1000,:].copy()
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
995,Toteteca Shoulder Bag (Beige) (TT2486Beige),"[Category : Fashion Accessories, Type : hand_m...",Toteteca,636
996,ADDA Men Big BOSS Black Silver Slippers UK-11,"[comfortable, washable, soft]",ADDA,810
997,"Etnies Women's Fader ls W's, Black Pink, 5.5 M US",[Modern version of a long running fan favorite...,Etnies,997
998,"Zensual Kool Kajal, 2.5g",[OTHERS],Zensual,998


In [82]:
y = dataset_temp['BROWSE_NODE_ID'].values
y

array([  0,   1,   2,   3,   4,   5,   6,   7,   7,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  28,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  38,  42,  43,  44,  45,  46,   4,   4,   4,   4,  45,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,   4,  68,  69,  70,  38,  72,  55,  74,  75,  76,  77,
        78,  79,   0,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100,   3, 102, 103,
       104, 105, 106, 107, 108, 109, 110,  55, 112, 113, 114, 115, 116,
       117, 118,  55, 120, 121, 122,  82, 124, 125,  55, 127,   4, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139,  40, 133,  82,
       143,  69, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         4,   4,   4,   4, 160,   4,   4,   4,   4, 165,   4,   4,   4,
         4,   4,   4,   4,   4,   4,   4,  11, 133,  55, 179, 18

In [39]:
y = np.reshape(y.copy(), (1000, 1))
y

array([[  0],
       [  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  7],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 28],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 38],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [  4],
       [  4],
       [  4],
       [  4],
       [ 45],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [  4],
       [ 68],
       [ 69],
       [ 70],
      

In [83]:
y_enc = to_categorical(y.copy())

In [84]:
y_enc.shape

(1000, 999)

In [87]:
#DO not tamper
X_train, X_test, y_train_will_check, y_test_will_check = train_test_split(dataset_temp['TITLE'], y.copy(), shuffle = False, random_state = 0)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(dataset_temp['TITLE'], y_enc, shuffle = False, random_state = 0)

In [89]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(750,)
(250,)
(750, 999)
(250, 999)


In [90]:
xtrain_glove = [sent2vec(x) for x in tqdm(X_train.copy())]
xtest_glove = [sent2vec(x) for x in tqdm(X_test.copy())]

100%|██████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 2754.56it/s]


In [91]:
xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)

In [92]:
print(xtrain_glove.shape)
print(xtest_glove.shape)
print(y_train.shape)
print(y_test.shape)

(750, 300)
(250, 300)
(750, 999)
(250, 999)


In [93]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xtest_glove_scl = scl.transform(xtest_glove)

In [163]:
# ytrain_enc = np_utils.to_categorical(y_train)
# ytest_enc = np_utils.to_categorical(y_test)

In [94]:
model = Sequential()
print('Model Initialised')

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
print('3 layers added')

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
print('3 layers added')

model.add(Dense(999))
model.add(Activation('softmax'))
print('Softmax added')

# compile the model
print('Model compiling')
model.compile(loss='categorical_crossentropy', optimizer='adam')
print('DONE')

Model Initialised
3 layers added
3 layers added
Softmax added
Model compiling
DONE


In [95]:
model.fit(xtrain_glove_scl, y=y_train, batch_size=64, 
          epochs=2, verbose=2, 
          validation_data=(xtest_glove_scl, y_test))

Epoch 1/2
12/12 - 2s - loss: 7.0540 - val_loss: 6.8491
Epoch 2/2
12/12 - 0s - loss: 6.1662 - val_loss: 6.7466


In [96]:
y_pred = model.predict_classes(xtest_glove_scl)

c:\users\ayush.bajpayee\.conda\envs\amazon_hackathon\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [98]:
y_test_will_check

array([750, 751, 752, 753, 754, 755,  85, 757, 758, 759, 473, 761, 762,
       763, 764, 216, 766, 567, 768, 151, 770, 771, 285, 773, 773, 775,
       209, 777, 778, 709, 780, 781, 782, 615, 784, 785,   4, 787, 788,
       789,  89, 289,  55, 362, 794, 789, 796, 797, 798, 589, 800, 801,
       237, 803, 804, 805, 805,   4, 808,  55, 810, 811, 812, 509, 489,
        98, 816,  91, 818, 819,  22, 821, 409,  89, 824,  69, 826, 827,
       828, 829, 830,  75, 832,   4, 834,  92, 836, 837,  91, 839, 840,
       841, 842,  55, 651, 750, 846, 847, 840, 849, 850, 851, 852,   5,
       714, 855, 856, 251, 858, 411, 153, 534, 862, 863, 864, 110, 866,
        82, 868, 869, 653, 871, 872, 873, 714, 875, 876, 877, 878, 423,
       880, 881, 709, 883,   4, 205, 886, 887, 816, 663, 890, 891, 892,
       249, 134, 434, 896, 572, 530, 899, 900, 259, 471, 903, 532, 905,
       906, 245, 908,  69,  55, 911, 912, 272, 914, 125, 916, 917, 918,
       919, 876, 289, 922, 923, 924, 925, 926, 872, 928, 929, 85

In [97]:
y_pred

array([195,  40, 711, 135, 477, 209, 201,  31,  64, 506,  82, 289, 390,
       179, 413, 413,  55, 506, 189, 359, 135, 893, 154, 282,  84, 201,
       209, 242, 234, 421, 948, 543, 553, 311, 771,  10,   4, 365, 506,
       366,  89, 289, 714, 413, 187, 467, 506, 187, 471, 245, 939, 650,
        64, 477,  40, 477, 362, 384,  35,  55, 248, 180,  31, 289, 255,
       543, 289, 242, 277, 442, 187, 490,  23, 153, 506, 918, 477,  31,
       635, 365,  69, 417, 284,   4, 506, 684, 466, 187, 366,  39, 359,
       241, 506, 289, 209, 366, 746, 801, 365, 179,  89, 671, 477, 732,
        69, 676,  61,  55,  31, 413,  55, 711, 362, 289,  35, 506, 187,
        84, 365, 506,  64, 379, 477, 365, 362, 379, 289, 915, 506, 187,
       506, 241, 165, 100,   4,  55, 506, 390, 477,  69,  31, 750, 323,
        35, 708, 241,  40, 209, 522, 349, 276,  31, 506, 241, 861, 207,
       506, 245,  55,  69,  55, 708, 540, 187, 365,  92,  55, 788, 477,
       788, 379, 289,  55, 540, 255, 506, 247, 301, 465, 506, 10

In [77]:
%%time

#Train Accuracy
y_pred = model.predict(xtest_glove_scl)
print('accuracy %s' % accuracy_score(y_pred, y_train))

array([[0.00100771, 0.00159982, 0.00112739, ..., 0.00084192, 0.0005378 ,
        0.00143024],
       [0.00118619, 0.0016276 , 0.00096432, ..., 0.0009725 , 0.00076593,
        0.00197639],
       [0.00086341, 0.00112254, 0.00102953, ..., 0.00110695, 0.00095087,
        0.00121763],
       ...,
       [0.0012835 , 0.00161267, 0.00072695, ..., 0.00106035, 0.00089834,
        0.00104407],
       [0.00138175, 0.00111768, 0.00120819, ..., 0.00101625, 0.00088383,
        0.00100281],
       [0.00089691, 0.00125742, 0.00098179, ..., 0.0011121 , 0.00098312,
        0.00116523]], dtype=float32)

In [99]:
%%time

# #Test Accuracy
# y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test_will_check))

accuracy 0.06
Wall time: 12 ms
